# Web Scrapping y análisis predictivo

A continuación realizo un sencillo scrapping de una tabla de wikipedia sobre el Índice de Desarrollo Humano (https://es.wikipedia.org/wiki/Anexo:Pa%C3%ADses_de_Am%C3%A9rica_Latina_por_%C3%ADndice_de_desarrollo_humano). En ese sentido, detallo cada parte del proceso de limpieza de los datos y lo convierto en un dataframe listo para analizar.

Finalmente realizo un proceso de validación por K-Fold y la predicción con ayuda de una sencilla regresión lineal. Ahora bien, este tipo de datos tiene un mejor desempeño en cuanto a su poder predectivo si se trabajan metodologías de series de tiempo tales como ARIMA, ARMA, GARCH, entre otras. Posteriormente, actualizaré el Notebook con este análisis.

In [1]:
! pip install beautifulsoup4

Comenzamos descargando y definiendo el script de HTML sobre el que haremos el scrapping. 

In [2]:
from bs4 import BeautifulSoup
import requests

pagina = requests.get('https://es.wikipedia.org/wiki/Anexo:Pa%C3%ADses_de_Am%C3%A9rica_Latina_por_%C3%ADndice_de_desarrollo_humano')  # Con requests podemos descargarnos el html de una página
soup = BeautifulSoup(pagina.content, 'html.parser')  # Convertimos el contenido de una página utilizando un transformador

print(soup.prettify())  # Imprimimos una versión más estética del html

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="es">
 <head>
  <meta charset="utf-8"/>
  <title>
   Anexo:Países de América Latina por índice de desarrollo humano - Wikipedia, la enciclopedia libre
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":[",\t."," \t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","enero","febrero","marzo","abril","mayo","junio","julio","agosto","septiembre","octubre","noviembre","diciembre"],"wgRequestId":"a584c749-8231-419c-93bf-62e5c21c889a","wgCSPNonce":!1,"wgCanonicalNamespace":"Anexo","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":104,"wgPageName":"Anexo:Países_de_América_Latina_por_índice_de_desarrollo_humano","wgTitle":"Países de América Latina por índice de desarrollo humano","wgCurRevisionId":138816404,"wgRevisionId":138816404,"wgArticleId":7260376,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUser

A continuación realizamos el filtrado de datos, teniendo como punto de partida las etiquetas 'table' que tengan como atributo 'class' el especificado aquí debajo.

In [3]:
tabla = soup.find('table',{'class':'wikitable sortable'}) 
tabla

<table class="wikitable sortable">
<tbody><tr>
<th>País
</th>
<th>2018
</th>
<th>2015
</th>
<th>2010
</th>
<th>2005
</th>
<th>2000
</th>
<th>1995
</th>
<th>1990
</th>
<th>Cambio
</th>
<th>Porcentaje
</th></tr>
<tr>
<td><span class="flagicon"><img alt="Bandera de Chile" class="thumbborder" data-file-height="1000" data-file-width="1500" decoding="async" height="13" src="//upload.wikimedia.org/wikipedia/commons/thumb/7/78/Flag_of_Chile.svg/20px-Flag_of_Chile.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/7/78/Flag_of_Chile.svg/30px-Flag_of_Chile.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/7/78/Flag_of_Chile.svg/40px-Flag_of_Chile.svg.png 2x" title="Bandera de Chile" width="20"/></span><a href="/wiki/Chile" title="Chile">Chile</a>
</td>
<td>0,847
</td>
<td>0,839
</td>
<td>0,800
</td>
<td>0,788
</td>
<td>0,753
</td>
<td>0,726
</td>
<td>0,703
</td>
<td>0,144
</td>
<td>20.48%
</td></tr>
<tr>
<td><span class="flagicon"><img alt="Bandera de Argentina" class="th

Con ayuda de Pandas usamos el método 'read_html', el cual partiendo del string del elemento anteriormente filtrado, nos devuelve una lista con el contenido de la tabla

In [4]:
import pandas as pd
df = pd.read_html(str(tabla))

In [5]:
df

[                    País  2018  2015  2010  ...  1995  1990  Cambio  Porcentaje
 0                  Chile   847   839   800  ...   726   703     144      20.48%
 1              Argentina   830   828   818  ...   731   707     123      17.39%
 2                Uruguay   808   802   774  ...   710   692     116      16.76%
 3             Costa Rica   795   782   758  ...   688   659     136      20.63%
 4                   Cuba   794   786   754  ...   686   655     139      21.22%
 5                 Panamá   779   768   777  ...   654   676     102      15.08%
 6                 México   767   759   739  ...   672   652     115      17.63%
 7                 Brasil   761   755   726  ...   650   613     148      24.14%
 8               Colombia   761   753   729  ...   633   599     162      27.04%
 9              Guatemala   745   733   700  ...   624   593     152      25.63%
 10  República Dominicana   726   763   753  ...   661   638      88      13.79%
 11              Paraguay   

Convertimos esta lista en un objeto array de numpy para facilitar su conversión a dataframe, pero en este punto podemos hacer uso de la estrategia que nos apatezca.

In [6]:
import numpy as np

data = np.array(df)
data

array([[['Chile', 847, 839, 800, 788, 753, 726, 703, 144, '20.48%'],
        ['Argentina', 830, 828, 818, 777, 770, 731, 707, 123, '17.39%'],
        ['Uruguay', 808, 802, 774, 756, 742, 710, 692, 116, '16.76%'],
        ['Costa Rica', 795, 782, 758, 739, 719, 688, 659, 136, '20.63%'],
        ['Cuba', 794, 786, 754, 727, 711, 686, 655, 139, '21.22%'],
        ['Panamá', 779, 768, 777, 730, 686, 654, 676, 102, '15.08%'],
        ['México', 767, 759, 739, 729, 705, 672, 652, 115, '17.63%'],
        ['Brasil', 761, 755, 726, 699, 684, 650, 613, 148, '24.14%'],
        ['Colombia', 761, 753, 729, 692, 662, 633, 599, 162, '27.04%'],
        ['Guatemala', 745, 733, 700, 675, 653, 624, 593, 152, '25.63%'],
        ['República Dominicana', 726, 763, 753, 714, 672, 661, 638, 88,
         '13.79%'],
        ['Paraguay', 730, 718, 692, 667, 640, 622, 588, 136, '23.12%'],
        ['Bolivia', 703, 685, 656, 632, 616, 578, 540, 163, '30.18%'],
        ['Venezuela', 667, 660, 659, 638, 608, 573, 529

In [7]:
result = data[0, :, :] 
# Aquí eliminamos una de las tres dimensiones del array, ya que de otra forma no será posible convertirlo a dataframe

Creamos nuestro dataframe y colocamos las nomenclaturas de acuerdo al orden de la tabla original. 

Encontramos que la conversión le quito los ceros iniciales que junto con el punto hacían la separación decimal. Además encontramos que todas las variables son entendidas como objetos y en el caso particular de la variable 'porcentaje' tenemos que cada número viene acompañado de un signo porcentual que no deseamos. Por lo que posteriormente corregiremos estos problemas.

In [8]:
indiceDesarrolloHumano = pd.DataFrame(result, columns=['Paises','IDH_2018','IDH_2015','IDH_2010','IDH_2005','IDH_2000','IDH_1995','IDH_1990','Cambio','Porcentaje'])

In [9]:
indiceDesarrolloHumano

,Paises,IDH_2018,IDH_2015,IDH_2010,IDH_2005,IDH_2000,IDH_1995,IDH_1990,Cambio,Porcentaje
0,Chile,847,839,800,788,753,726,703,144,20.48%
1,Argentina,830,828,818,777,770,731,707,123,17.39%
2,Uruguay,808,802,774,756,742,710,692,116,16.76%
3,Costa Rica,795,782,758,739,719,688,659,136,20.63%
4,Cuba,794,786,754,727,711,686,655,139,21.22%
5,Panamá,779,768,777,730,686,654,676,102,15.08%
6,México,767,759,739,729,705,672,652,115,17.63%
7,Brasil,761,755,726,699,684,650,613,148,24.14%
8,Colombia,761,753,729,692,662,633,599,162,27.04%
9,Guatemala,745,733,700,675,653,624,593,152,25.63%


In [10]:
indiceDesarrolloHumano.set_index('Paises', inplace=True)  # Establecemos como índice a la variable 'Paises'

Para solucionar lo anteriormente descrito procederemos a realizar un loop que convierta de objeto a string, de forma tal que podamos trabajar de forma sencilla sobre estos datos.

Después, agregaremos un cero a la izquierda a cada observación de interés (los datos de las columnas especificadas en el loop).

Posteriormente, con ayuda de una función que aquí definimos como 'insertar_punto' ponemos un punto entre el cero que agregamos y el resto del número. Y junto con el loop hacemos que cada número que había perdido su separación decimal la recupere.

Por último, convertimos a númericos los strings que habíamos modificado y corregido.

In [11]:
def insertar_punto(string):
    return string[:1] + '.' + string[1:]

In [12]:
N = 1
for col in indiceDesarrolloHumano.columns[0:8]:
  indiceDesarrolloHumano[col] = indiceDesarrolloHumano[col].astype(str)
  indiceDesarrolloHumano[col] = [i.zfill(N + len(i)) for i in indiceDesarrolloHumano[col]]
  indiceDesarrolloHumano[col] = indiceDesarrolloHumano[col].map(insertar_punto)
  indiceDesarrolloHumano[col] = pd.to_numeric(indiceDesarrolloHumano[col], errors='coerce')

Aquí corregiremos el caso especial de la variable 'Porcentaje', la cual tenía un signo '%' indeseable. No realizamos ninguna transformación adicional, pero si fuese de interés podemos convertilo a notación decimal acotado entre 0 y 1.

In [13]:
# Transformación a la columna 'porcentaje'

indiceDesarrolloHumano['Porcentaje'] = [c.replace('%', '') for c in indiceDesarrolloHumano['Porcentaje']]
indiceDesarrolloHumano['Porcentaje'] = pd.to_numeric(indiceDesarrolloHumano['Porcentaje'], errors='coerce')

In [14]:
indiceDesarrolloHumano

,IDH_2018,IDH_2015,IDH_2010,IDH_2005,IDH_2000,IDH_1995,IDH_1990,Cambio,Porcentaje
Paises,,,,,,,,,
Chile,0.847,0.839,0.800,0.788,0.753,0.726,0.703,0.144,20.48
Argentina,0.830,0.828,0.818,0.777,0.770,0.731,0.707,0.123,17.39
Uruguay,0.808,0.802,0.774,0.756,0.742,0.710,0.692,0.116,16.76
Costa Rica,0.795,0.782,0.758,0.739,0.719,0.688,0.659,0.136,20.63
Cuba,0.794,0.786,0.754,0.727,0.711,0.686,0.655,0.139,21.22
Panamá,0.779,0.768,0.777,0.730,0.686,0.654,0.676,0.102,15.08
México,0.767,0.759,0.739,0.729,0.705,0.672,0.652,0.115,17.63
Brasil,0.761,0.755,0.726,0.699,0.684,0.650,0.613,0.148,24.14
Colombia,0.761,0.753,0.729,0.692,0.662,0.633,0.599,0.162,27.04


In [15]:
indiceDesarrolloHumano.dtypes

IDH_2018      float64
IDH_2015      float64
IDH_2010      float64
IDH_2005      float64
IDH_2000      float64
IDH_1995      float64
IDH_1990      float64
Cambio        float64
Porcentaje    float64
dtype: object

## Modelado

En este punto se realizará un modelo sencillo de regresión lineal para predecir el resultado del IDH para 2020, pero aclaro que no es el procedimiento acertado toda vez que al ser datos de serie de tiempo tenemos que tener presentes conceptos como estacionariedad, autocorrelación, entre otros. Por lo que un futuro pretendo abordar con mayor profundidad este análisis.

Modelos sugeridos: ARMA - ARIMA - GARCH - entre otros.

Igualmente emplearé la técnica de validación K-Fold, la cual correrá 4 modelos para analizar cómo se comporta el modelo a medida que vamos cambiando tanto el conjunto de entrenamiento como el de prueba

In [34]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import r2_score
from sklearn import linear_model

In [38]:
indiceDesarrolloHumano.reset_index(inplace=True)

In [44]:
X = indiceDesarrolloHumano.iloc[:,3:8]
Y = indiceDesarrolloHumano.iloc[:,2]

A continuación establecemos la validación por K-Fold partiendo los datos cuatro veces entre el conjunto de prueba y el de entrenamiento. En ese sentido, establecemos un loop que vaya iterando por cada conjunto los diferentes particiones de los datos y almacenamos en una lista llamada 'Resultados' los diferentes valores de R cuadrado y calculamos la media de estos valores.

In [46]:
kf = KFold(n_splits=4)
kf.get_n_splits(X)

regresion = linear_model.LinearRegression()

resultados = []

for train_index, test_index in kf.split(X):
  X_train, X_test = X.loc[train_index,],X.loc[test_index]
  y_train, y_test = Y[train_index], Y[test_index]
  regresion.fit(X_train,y_train)
  predicciones = regresion.predict(X_test)
  print('R2: ',r2_score(y_test, predicciones))
  resultados.append(r2_score(y_test, predicciones))

print('R2 medio: ', np.mean(resultados))

R2:  -0.047325914627248444
R2:  -2.635323148534786
R2:  0.779599899993139
R2:  0.9244935958861276
R2 medio:  -0.24463889182069198


Un detalle a analizar de lo anterior es que los datos no se ajustan de forma correcta para cada uno de los modelos de regresión que corrimos. Esto porque en 2 regresiones tuvimos valores negativos de R cuadrado y en un caso tuvimos un R cuadrado que se salió de la cota 0-1.

En ese sentido, es clave recalcar que este no es el mejor modelo para hacer predicciones, pero por fines de simplicidad y con la meta de aplicar de forma rápida la validación K-Fold se realizó así.

Posteriormente vamos a correr el modelo sin realizar validación de datos, usándolos con la meta de predecir el IDH de 2025

In [47]:
regresion = linear_model.LinearRegression()
regresion.fit(X, Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [49]:
r2_score(Y, regresion.predict(X))

0.9850372116245699

In [50]:
regresion.coef_

array([ 0.61331468,  0.12270571,  0.25579375,  0.27884626, -0.27767104])

In [53]:
regresion.intercept_

0.03660312109745545

Aquí multiplicamos los valores de cada columna con los coeficiente de la anterior regresión para hallar la predicción de 2025. Esto partiendo del hecho de que teníamos datos periódicos de 5 años.

In [51]:
np.sum(df.iloc[:,2:7] * regresion.coef_, axis=1) # Predicciones 2025

Paises
Chile                   0.787196
Argentina               0.801518
Uruguay                 0.763103
Costa Rica              0.748349
Cuba                    0.742930
Panamá                  0.736255
México                  0.729370
Brasil                  0.717038
Colombia                0.711539
Guatemala               0.688521
República Dominicana    0.728494
Paraguay                0.680138
Bolivia                 0.648684
Venezuela               0.650874
El Salvador             0.587592
Nicaragua               0.603298
Honduras                0.586751
Haití                   0.457301
Latinoamérica           0.687589
dtype: float64